<a href="https://colab.research.google.com/github/itzThillaiC/OCR-table-data-extraction-/blob/main/Bank_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd 
import tensorflow as tf

In [2]:
#!pip install paddlepaddle-gpu==2.3.2 cudatoolkit==10.2

In [3]:
!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


# pdf2image

## Installation

In [4]:
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

## Conversion

In [5]:
from pdf2image import convert_from_path

In [7]:
images = convert_from_path('citibankstatement.pdf')

In [8]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [9]:
for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg', 'JPEG')

# Layout

## Installation

In [10]:
#!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [11]:
a=5

In [12]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

--2022-12-16 16:39:32--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl.1’

layoutparser-0.0.0- 100%[===================>]  18.26M  3.66MB/s    in 14s     

2022-12-16 16:39:48 (1.31 MB/s) - ‘layoutparser-0.0.0-py3-none-any.whl.1’ saved [19145360/19145360]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./layoutparser-0.0.0-py3-none-any.whl
layoutparser is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


## Table Extraction

In [25]:
import cv2
import layoutparser as lp
image = cv2.imread("/content/pages/page1.jpg")

image = image[..., ::-1]

# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.5,
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library
# detect
layout = model.detect(image)

In [26]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=124.35540771484375, y_1=443.31475830078125, x_2=1532.647216796875, y_2=1689.260009765625), text=None, id=None, type=Table, parent=None, next=None, score=0.9693410396575928)], page_data={})

In [27]:
x_1=0
y_1=0
x_2=0
y_2=0

for l in layout:
  #print(l)
  if l.type == 'Table':
    x_1 = int(l.block.x_1)
    print(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)
    
    break

124.35541


In [28]:
print(x_1,y_1,x_2,y_2)

124 443 1532 1689


In [29]:
im = cv2.imread('/content/pages/page1.jpg')

In [30]:
cv2.imwrite('ext_im.jpg', im[y_1:y_2,x_1:x_2])

True

# Text Detection and Recognition

In [31]:
from paddleocr import PaddleOCR, draw_ocr

In [32]:
ocr = PaddleOCR(lang='en')
image_path = '/content/ext_im.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)[0]

[2022/12/16 16:41:59] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [37]:
print(output)

[[[[3.0, 43.0], [54.0, 43.0], [54.0, 65.0], [3.0, 65.0]], ('03/05', 0.9997844696044922)], [[[147.0, 45.0], [267.0, 45.0], [267.0, 65.0], [147.0, 65.0]], ('17692819624', 0.9985954761505127)], [[[361.0, 45.0], [500.0, 45.0], [500.0, 65.0], [361.0, 65.0]], ('Adani Electricity', 0.9833509922027588)], [[[537.0, 42.0], [635.0, 45.0], [635.0, 67.0], [537.0, 63.0]], ('www.adan', 0.9958149790763855)], [[[1329.0, 45.0], [1404.0, 45.0], [1404.0, 68.0], [1329.0, 68.0]], ('7561.95', 0.9990621209144592)], [[[3.0, 97.0], [54.0, 97.0], [54.0, 118.0], [3.0, 118.0]], ('03/05', 0.9997413754463196)], [[[147.0, 98.0], [265.0, 98.0], [265.0, 118.0], [147.0, 118.0]], ('17695770402', 0.998777449131012)], [[[359.0, 98.0], [500.0, 98.0], [500.0, 118.0], [359.0, 118.0]], ('Adani Electricity', 0.9741377830505371)], [[[539.0, 96.0], [634.0, 100.0], [633.0, 120.0], [538.0, 117.0]], ('www.adan', 0.9889125227928162)], [[[1330.0, 97.0], [1402.0, 97.0], [1402.0, 118.0], [1330.0, 118.0]], ('6866.25', 0.9994797110557556)

In [39]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]

In [40]:
image_boxes = image_cv.copy()


In [41]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [42]:
cv2.imwrite('detections.jpg', image_boxes)

True

# Reconstruction

## Get Horizontal and Vertical Lines

In [43]:
im = image_cv.copy()

In [44]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)
  

In [45]:
cv2.imwrite('horiz_vert.jpg',im)

True

## Non-Max Suppression

In [46]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [47]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[ 0  5 10 18 20 24 28 34 35 40 45 52 57 58 63 68 69 72 75 78 87 89 93]


In [48]:
im_nms = image_cv.copy()

In [49]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
  

In [50]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [51]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [52]:
print(vert_out)

tf.Tensor([34 78 28 81 12], shape=(5,), dtype=int32)


In [53]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[12 28 34 78 81]


In [54]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)
  

In [55]:
cv2.imwrite('im_nms.jpg',im_nms)

True

## Convert to CSV

In [56]:


out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

(23, 5)
[['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', '']]


In [57]:

unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

[358, 0, 518, 1246]
[145, 0, 267, 1246]
[1327, 0, 1405, 1246]
[3, 0, 54, 1246]
[542, 0, 621, 1246]


In [58]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

[3 1 0 4 2]


In [59]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [60]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0
      
  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))
  
  return inter / float(box_1_area + box_2_area - inter)

In [61]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [62]:
out_array=np.array(out_array)

In [63]:
out_array

array([['03/05', '17692819624', 'Adani Electricity', 'www.adan',
        '7561.95'],
       ['03/05', '17695770402', 'Adani Electricity', 'www.adan',
        '6866.25'],
       ['16/05', '48214192783', 'ADANIELECMUM', 'MUMBAI', '9019.56'],
       ['18/05', '00404901886', 'CIT*McAfee Ireland Lim Netherla',
        'CIT*McAfee Ireland Lim Netherla', '3499.00'],
       ['24/05', '00067144003', 'TOP DOGHEALTHCARE PVT -005/012',
        'TOP DOGHEALTHCARE PVT -005/012', '655.00'],
       ['24/05', '15347144003', 'TOP DOG HEALTHCARE PVT-005/012',
        'TOP DOG HEALTHCARE PVT-005/012', '9996.00'],
       ['24/05', '30043144003', 'BAFNA MEDICALand GEN-005/012',
        'BAFNA MEDICALand GEN-005/012', '774.00'],
       ['24/05', '37633144003', 'ENHANCED PAYLITEEMI-014/036',
        'ENHANCED PAYLITEEMI-014/036', '9822.00'],
       ['24/05', '41078144003', 'Pasfar Technologies', '-005/012',
        '469.00'],
       ['24/05', '56476144003', 'BAFNAMEDICALand GEN-005/012',
        'BAFNAMEDICAL

In [64]:
pd.DataFrame(out_array).to_csv('sample.csv')

## Merging Cells

In [65]:
current_bank=['']*len(out_array[0,:])
print(current_bank)

['', '', '', '', '']


In [66]:
def empty(arr):
  for i in arr:
    if i=='':
      return True
  return False

In [67]:
cleaned_array=[]

In [68]:
for i in range(len(out_array)):
  if not empty(out_array[i]):
    current_bank=[out_array[i][j] for j in range(len(out_array[i]))]
    cleaned_array.append(current_bank)
    not_empty=True
  else:
    for j in range(len(out_array[i])):
      current_bank[j]+=' '+out_array[i][j]
    print('-->',current_bank)
cleaned_array=np.array(cleaned_array)
print(cleaned_array)

--> ['24/05 24/05', '99913144003 ', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL)', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL)', '971.00 6638.45']
--> ['24/05 24/05 24/05', '99913144003  ', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL) REVERSALOFIGST@ 18%', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL) REVERSALOFIGST@ 18%', '971.00 6638.45 1673.38CR']
--> ['24/05 24/05 24/05 24/05', '99913144003   ', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL) REVERSALOFIGST@ 18% IGST @ 18%', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL) REVERSALOFIGST@ 18% ', '971.00 6638.45 1673.38CR 1994.52']
--> ['24/05 24/05 24/05 24/05 Addit', '99913144003    ional Card', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL) REVERSALOFIGST@ 18% IGST @ 18% =>XXXX XXXX XXXX 5529', 'Galaxy cloud kitchens -005/006 INTERESTCHARGE@3.75%(45.0%ANNUAL) REVERSALOFIGST@ 18%  =>XXXX XXXX XX

In [69]:
pd.DataFrame(cleaned_array).to_csv('cleaned.csv')

# Convert to OWL Format

# **CSV to Text**

In [70]:
import  jpype     
import  asposecells  


jpype.startJVM() 
from asposecells.api import Workbook

workbook = Workbook("/content/sample.csv")
workbook.save("Output.docx")
jpype.shutdownJVM()

ModuleNotFoundError: ignored

# Brighten Image (For Anyone dealing with PDFs created from scanned images)

In [ ]:
from PIL import Image, ImageEnhance

#read the image
im = Image.open("ext_im.jpg")

#image brightness enhancer
enhancer = ImageEnhance.Brightness(im)

factor = 1 #gives original image
im_output = enhancer.enhance(factor)
im_output.save('ext_im-1.jpg')

factor = 1.5## brightens the image
im_output = enhancer.enhance(factor)
im_output.save('ext_im-2.jpg')
